<a href="https://colab.research.google.com/github/okada-t-rafael/pytorch_study/blob/master/05_pytorch_going_modular_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises

In [1]:
import os

os.makedirs("going_modular", exist_ok=True)

## Exercise 1
Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.

* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [2]:
%%writefile going_modular/get_data.py
"""
Contains functions for downloading an image dataset and unzip it.
"""
import logging
import requests
import zipfile
from pathlib import Path
from typing import Tuple


def download_zip_dataset(
        dataset_url: str,
        folder_name: str,
        ) -> Tuple[Path, bool]:
    """Downloads an image dataset from a given url and unzip it.

    The files within the zip must respect the following structure:
    .
    |-- root_folder
        |-- test
        |   |-- class_one
        |   |-- class_two
        `-- train
            |-- class_one
            |-- class_two

    Args:
        dataset_url: An URL of a zip file containing images for each class
            divided into train and test subfolders.
        folder_name: The name of the folder to be creatd when unzipping the
            zip files.

    Returns:
        The path where the image were download and a boolean indicating whether
        everything was executed as expected. For example:

        (PosixPath("folder_name"), True)

        If last value within the return is a False, should not use the other
        values.
    """
    # Setup path to a data folder
    data_path: Path = Path("data")
    image_path: Path = data_path / folder_name

    # Check whether the image folder already exists, if now download it...
    if image_path.is_dir():
        logging.info(
            f"Directory '{image_path}' already exists. Skipping download.")
        return image_path, True

    logging.info(f"Creating directory: '{image_path}'")
    image_path.mkdir(parents=True, exist_ok=True)

    # Download zipfile
    zipfile_name = dataset_url.split("/")[-1]
    try:
        logging.info(f"Downloading: '{zipfile_name}'")
        req = requests.get(dataset_url)
    except requests.exceptions.RequestException as e:
        logging.error(f"Error while downloading '{dataset_url}': {e}")
        return Path(""), False

    # Saving downloaded file
    try:
        with open(data_path / zipfile_name, "wb") as f:
            f.write(req.content)
    except IOError as e:
        logging.error(f"Error while writing '{zipfile_name}': {e}")
        return Path(""), False

    # Unzip images
    try:
        with zipfile.ZipFile(data_path / zipfile_name, "r") as zip_ref:
            logging.info(f"Unzipping: '{zipfile_name}'")
            zip_ref.extractall(image_path)
    except Exception as e:
        logging.error(f"Error while unzipping '{zipfile_name}': {e}")
        return Path(""), False

    return image_path, True


Writing going_modular/get_data.py


In [3]:
import logging
from going_modular.get_data import download_zip_dataset


# Setting logging level.
logging.getLogger().setLevel(logging.INFO)

# Some definitions
DATASET_URL = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip"
WORK_FOLDER = "pizza_steak_sushi"

# Donwload and unzip images.
image_path, _ = download_zip_dataset(DATASET_URL, WORK_FOLDER)

INFO:root:Creating directory: 'data/pizza_steak_sushi'
INFO:root:Downloading: 'pizza_steak_sushi_20_percent.zip'
INFO:root:Unzipping: 'pizza_steak_sushi_20_percent.zip'


## Exercise 2
Use Python's `argparse` modeule to be able to send the train.py custom hyperparameter values for training procedures.

* Add an argument flag for using a different:
    * Training/testing directory
    * Learning rate
    * Batch size
    * Number of epochs to train for
    * Number of hidden units in the TinyVGG model
        * Keep the default values for each of the above arguments as what they already are (as in notebook 05)
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 --batch_size 64 --num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the going_modular forder of the course GitHub.

In [4]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader for image classification
data.
"""
import logging
import os
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from typing import List, Tuple


# Default number of workers for DataLoader
NUM_WORKERS = os.cpu_count()


class ErrorDataset(Dataset):
    """Dummy Dataset for creating a ZeroValue instance for DataLoaders."""
    def __len__(self):
        return 0


def create_dataloaders(
        image_path: Path,
        train_transform: transforms.Compose,
        test_transform: transforms.Compose,
        batch_size: int=32,
        num_workers: int=NUM_WORKERS
        ) -> Tuple[Tuple[DataLoader, DataLoader], List[str], bool]:
    """Creates training and testing DataLoaders.

    Takes in a image path directory and turns them into PyTorch Datasets and
    then into PyTorch DataLoaders.

    Args:
        image_path: Location where the train and test forlder are located.
        train_transform: Transformations to be applied to the train images.
        test_transform: Transformations to be applied to the test images.
        batch_size: Size of the batches to be created for the dataloaders.

    Returns:
        A tuple of dataloaders for the train and test datasets, a list
        containing the names of the images labels, and a bool indicating
        whether the function was executed as expected.

        If last value within the return is a False, should not use the other
        values.
    """
    # Create datasets
    logging.info("Creating Datasets.")

    if not image_path.is_dir():
        logging.error(f"There is no folder: '{image_path}'.")
        err_dataloader = DataLoader(ErrorDataset())
        return ((err_dataloader, err_dataloader), [], False)

    try:
        train_dataset = datasets.ImageFolder(
            root=image_path / "train",
            transform=train_transform)
        test_dataset = datasets.ImageFolder(
            root=image_path / "test",
            transform=test_transform)
        image_classes_list = train_dataset.classes

    except Exception as e:
        logging.error(f"Error loading images from: '{image_path}'.")
        err_dataloader = DataLoader(ErrorDataset())
        return ((err_dataloader, err_dataloader), [], False)

    # Turn datasets into dataloaders
    logging.info("Turning train and test datasets into dataloaders.")

    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True,
        pin_memory=True)

    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,
        pin_memory=True)

    return ((train_dataloader, test_dataloader), image_classes_list, True)


Writing going_modular/data_setup.py


In [5]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple


def train_step(
        model: torch.nn.Module,
        dataloader: torch.utils.data.DataLoader,
        loss_fn: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
        device: str
        ) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then runs through all the
    required training steps (forward pass, loss calculation, optimizer step).

    Args:
        model: A Pytorch model to be trained.
        dataloader: A Dataloader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A tuple of training loss and training accuracy metrics. In the form
        (train_loss, train_accuracy). For example: (0.1112, 0.8743)
    """
    # Put the model into target device
    model.to(device)

    # Pu the model in train mode
    model.train()

    # Setup train loss and train accuracy variables
    train_loss: float = 0.0
    train_acc: float = 0.0

    # Loop through dataloader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to the target device
        X = X.to(device)
        y = y.to(device)

        # 1. Forward pass
        y_pred_logits = model(X)  # output model logits

        # 2. Calculate the loss
        loss = loss_fn(y_pred_logits, y)
        train_loss += loss.item()  # convert this tensor to a standard python number

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate accuracy metric
        y_pred_probs = torch.softmax(y_pred_logits, dim=1)
        y_pred_labels = torch.argmax(y_pred_probs, dim=1)
        train_acc += (y_pred_labels == y).sum().item() / len(y_pred_labels)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc


def test_step(
        model: torch.nn.Module,
        dataloader: torch.utils.data.DataLoader,
        loss_fn: torch.nn.Module,
        device: str) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to 'eval' mode and then performs a forward
    pass on a testing dataset.

    Args:
        model: A Pytorch model to be tested.
        dataloader: A Dataloader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A tuple of testing loss and testing accuracy metrics. In the form
        (test_loss, test_accuracy). For example: (0.1112, 0.8743)
    """
    # Put model into target device
    model.to(device)

    # Put model in eval mode
    model.eval()

    # Step test loss and test accuracy variables
    test_loss: float = 0.0
    test_acc: float = 0.0

    # Turn on inference mode
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X = X.to(device)
            y = y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate the loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate the accuracy
            test_pred_probs = test_pred_logits.softmax(dim=1)
            test_pred_labels = test_pred_probs.argmax(dim=1)
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

        # Adjust metrics to get average loss and accuracy per batch
        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)

        return test_loss, test_acc


def train(
        model: torch.nn.Module,
        train_dataloader: torch.utils.data.DataLoader,
        test_dataloader: torch.utils.data.DataLoader,
        optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module,
        epochs: int,
        device: str,
        ) -> Dict[str, List[float]]:
    """
    Trains and tests a PyTorch model.

    Passes a target model through 'train_step' and 'test_step' functions for a
    number of epochs, training and testing the model in the same epoch loop.
    Calculates, prints and stores evaluation metrics throughout.

    Args:
        model: A PyTorch model to be trained and tested.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        loss_fn: A PyTorch loss function to calculate on both datasets.
        epochs: An integer indicating how many epochs to train for.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A dictionary of training and testing loss as well as training and
        testing accuracy metrics. Each metric has a value in a list for each
        epoch.

        For example if training for epochs = 2: {
            train_loss: [2.0616, 1.0537],
            train_acc: [0.3945, 0.3945],
            test_loss: [1.2641, 1.5706],
            test_acc: [0.3400, 0.2973]
        }
    """
    # Create an empty results dictionary
    results: Dict[str, List[float]] = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []}

    # Loop through training and testing steps for a number o epochs
    for epoch in tqdm(range(epochs)):
        # Training step
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device)

        # Test step
        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)

        # Print out what's happening
        print(
            f" - Epoch: {epoch} | "
            f"Train_loss: {train_loss:.4f}, Train_acc: {train_acc:.3f}% | "
            f"Test_loss: {test_loss:.4f}, Test_acc: {test_acc:.3f}%")

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results


Writing going_modular/engine.py


In [6]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn
from typing import Tuple


class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN exaplainer website in
    Pytorch. See more: https://poloclub.github.io/cnn-explainer/
    """
    def __init__(
            self,
            in_channels: int,
            hidden_units: int,
            out_features: int,
            img_shape: Tuple[int, int]
            ) -> None:
        super().__init__()

        # First block
        self.conv_block_1, adj_img_shape = TinyVGG._create_conv_block(
            in_channels=in_channels,
            out_channels=hidden_units,
            img_shape=img_shape)

        # Second block
        self.conv_block_2, adj_img_shape = TinyVGG._create_conv_block(
            in_channels=hidden_units,
            out_channels=hidden_units,
            img_shape=adj_img_shape)

        # Classifier
        self.classifier = TinyVGG._create_classifier_block(
            in_features=hidden_units * adj_img_shape[0] * adj_img_shape[1],
            out_features=out_features)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))


    def _create_conv_block(
            in_channels: int,
            out_channels: int,
            img_shape: Tuple[int, int]
            ) -> Tuple[nn.Sequential, Tuple[int, int]]:
        """Creates a block of the neural network.

        This block includes two Conv2d and one MaxPool2d. And it also
        calculates the adjusted size of the 'image' for the flatten layer.

        Args:
            in_channels: The input size channel of the first conv layer.
            out_channels: The output size channel of the second conv layer.
            img_shape: The shape of the 'image' after passing through this
                block. It is influenced by the Conv2d layers and the MaxPool2d
                layer (see documentatin to adjust accordinly).

        Return:
            The instance of the block and the adjusted size of the 'image'.
        """
        # Create the conv block
        conv_block = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=1,
                padding=0),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=out_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=1,
                padding=0),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2,
                stride=2))

        # Calculate the ajusted image shape
        img_x, img_y = img_shape
        img_x -= 2  # first Conv2d
        img_x -= 2  # second Conv2d
        img_x = int(img_x / 2)  # MaPool2d
        img_y -= 2  # first Conv2d
        img_y -= 2  # second Conv2d
        img_y = int(img_y / 2)  # MaPool2d

        return conv_block, (img_x, img_y)


    def _create_classifier_block(
            in_features: int,
            out_features: int,
            ) -> nn.Sequential:
        """Creates a classifier block to the neural network.

        It is composed by a flatten layer and a linear layer.

        Args:
            in_features: The number of features to be inserted in the linear
                layer. Note that this number must be ajusted to due to the
                flatten layer.
            out_feataures: Number of labels for the neural network to predict.

        Return:
            An instance of the block.
        """
        return nn.Sequential(
            nn.Flatten(),
            nn.Linear(
                in_features=in_features,
                out_features=out_features
            )
        )


Writing going_modular/model_builder.py


In [7]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import logging
import torch
from pathlib import Path


def save_model(
        model: torch.nn.Module,
        target_dir: str,
        model_name: str,
        ) -> bool:
    """Saves a PyTorch model to a target directory.

    Args:
        model: A target PyTorch model to save.
        target_dir: A directory for saving the model to.
        model_name: A filename for the saved model. Should include either
            ".pth" or ".pt" as the filename extension.

    Returns:
        A boolean indicating whether the save operation was executed without
        errors.
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    if not model_name.endswith((".pth", ".pt")):
        logging.error("Model's name should end with '.pt' or '.pth'.")
        return False
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    try:
        torch.save(obj=model.state_dict(), f=model_save_path)
    except Exception as e:
        logging.erro(f"Error while saving model: {e}")
        return False

    return True

Writing going_modular/utils.py


In [8]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import argparse
import data_setup
import engine
import get_data
import logging
import model_builder
import os
import torch
import utils
from pathlib import Path
from torchvision import transforms


# Create a parser
parser = argparse.ArgumentParser(description="Get some hyperparameters.")

# Add an arg for num_epochs
parser.add_argument(
    "--num_epochs",
    default=10,
    type=int,
    help="the number of epochs to train for")

# Add an arg for batch_size
parser.add_argument(
    "--batch_size",
    default=32,
    type=int,
    help="numer of samples per batch")

# Add an arg for hidden_units
parser.add_argument(
    "--hidden_units",
    default=10,
    type=int,
    help="number of hidden units in hidden layers")

# Add an arg for learning_rate
parser.add_argument(
    "--learning_rate",
    default=0.001,
    type=float,
    help="learning rate to use for model")

# Add an arg for the image path
parser.add_argument(
    "--image_path",
    default="data/pizza_steak_sushi",
    type=str,
    help="image path in standard image classification format"
)

# Get our arguments from the parser
args = parser.parse_args()

# Setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
IMAGE_PATH = Path(args.image_path)

logging.info(
    f"Training a model epochs: {NUM_EPOCHS} \| "
    f"batch size: {BATCH_SIZE} \| hidden units: {HIDDEN_UNITS} \| "
    f"learning rate: {LEARNING_RATE}")

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
train_data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64), antialias=True),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()])

test_data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64), antialias=True),
    transforms.ToTensor()])

# Create DataLoaders
dataloaders, class_name_list, ok = data_setup.create_dataloaders(
    image_path=IMAGE_PATH,
    train_transform=train_data_transform,
    test_transform=test_data_transform,
    batch_size=BATCH_SIZE)

# Create model
model = model_builder.TinyVGG(
    in_channels=3,
    hidden_units=HIDDEN_UNITS,
    out_features=len(class_name_list),
    img_shape=(64, 64))

# Set Loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=LEARNING_RATE)

# Training
engine.train(
    model=model,
    train_dataloader=dataloaders[0],
    test_dataloader=dataloaders[1],
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=NUM_EPOCHS,
    device=device)

# Save
utils.save_model(
    model=model,
    target_dir="models",
    model_name="tinyvgg_model.pth")

Writing going_modular/train.py


In [9]:
!python going_modular/train.py --num_epochs 50 --batch_size 32 --hidden_units 20 --learning_rate 0.00025

  0% 0/50 [00:00<?, ?it/s] - Epoch: 0 | Train_loss: 1.0971, Train_acc: 0.296% | Test_loss: 1.0980, Test_acc: 0.287%
  2% 1/50 [00:06<05:28,  6.71s/it] - Epoch: 1 | Train_loss: 1.0913, Train_acc: 0.317% | Test_loss: 1.0954, Test_acc: 0.281%
  4% 2/50 [00:10<04:13,  5.27s/it] - Epoch: 2 | Train_loss: 1.0929, Train_acc: 0.329% | Test_loss: 1.0845, Test_acc: 0.338%
  6% 3/50 [00:15<03:47,  4.83s/it] - Epoch: 3 | Train_loss: 1.0803, Train_acc: 0.431% | Test_loss: 1.0640, Test_acc: 0.500%
  8% 4/50 [00:20<03:47,  4.96s/it] - Epoch: 4 | Train_loss: 1.0720, Train_acc: 0.429% | Test_loss: 1.0243, Test_acc: 0.495%
 10% 5/50 [00:24<03:30,  4.67s/it] - Epoch: 5 | Train_loss: 1.0594, Train_acc: 0.460% | Test_loss: 0.9737, Test_acc: 0.533%
 12% 6/50 [00:29<03:32,  4.83s/it] - Epoch: 6 | Train_loss: 0.9962, Train_acc: 0.485% | Test_loss: 0.9385, Test_acc: 0.509%
 14% 7/50 [00:34<03:22,  4.72s/it] - Epoch: 7 | Train_loss: 1.0276, Train_acc: 0.433% | Test_loss: 0.9339, Test_acc: 0.601%
 16% 8/50 [00:38

## Exercise 3
Create a Python script to predict on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the prediction on a custom image section in notebook 04.
* You may also have to write code to load in a trained model.

In [24]:
%%writefile going_modular/predict.py
import argparse
import logging
import model_builder
import torch
import torchvision

# Creating a parser
parser = argparse.ArgumentParser()

# Get an image path
parser.add_argument(
    "--image",
    help="target image filepath to predict on")

# Get a model path
parser.add_argument(
    "--model_path",
    default="models/tinyvgg_model.pth",
    type=str,
    help="target model to use for prediction")

args = parser.parse_args()

# Setup class names
class_name_list = ["pizza", "steak", "sushi"]

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Get the image path
IMAGE = args.image
logging.info(f"Predicting on {IMAGE}")

# Load in the model
model = model_builder.TinyVGG(
    in_channels=3,
    hidden_units=20,
    out_features=len(class_name_list),
    img_shape=(64, 64))

model.load_state_dict(torch.load(args.model_path))

# Data preparation
image = torchvision.io.read_image(str(IMAGE)).type(torch.float32)
image = image / 255.0

transform = torchvision.transforms.Resize(size=(64, 64))
image = transform(image)

# Predict on image
model.eval()
with torch.inference_mode():
    # Send image to target device
    image = image.to(device)

    # Get pred logits
    pred_logits = model(image.unsqueeze(dim=0))
    pred_prob = torch.softmax(pred_logits, dim=1)
    pred_label = torch.argmax(pred_prob, dim=1)

print(f"Pred class: {class_name_list[pred_label]}, Pred prob: {pred_prob.max():.3f}")  # noqa: E501

Overwriting going_modular/predict.py


In [25]:
!python going_modular/predict.py --image data/pizza_steak_sushi/test/pizza/1555015.jpg

Pred class: pizza, Pred prob: 0.462
